In [69]:
import requests
import pandas as pd

import time

import os
from dotenv import load_dotenv

load_dotenv()

google_key = os.getenv("GOOGLE_BOOKS")
nyt_key = os.getenv("NYT")

In [70]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [71]:
df_cached = pd.read_csv('df_cached.csv')
df_cached['primary_isbn13'] = df_cached['primary_isbn13'].apply(str)

In [72]:
df_cached

,age_group,amazon_product_url,article_chapter_link,asterisk,author,book_image,book_image_height,book_image_width,book_review_link,book_uri,...,rank,rank_last_week,sunday_review_link,title,updated_date,weeks_on_list,category,published_date,Unnamed: 0,date_datetime
0,NaN,https://www.amazon.com/dp/B0DTYKCJC9?tag=thene...,NaN,0,Jake Tapper and Alex Thompson,https://static01.nyt.com/bestsellers/images/97...,500,329,NaN,nyt://book/786e31c1-bc83-50f1-bd8d-995926458641,...,1,0,NaN,ORIGINAL SIN,2025-06-02T23:13:46.798Z,1,Political Science,2025-06-08,NaN,NaN
1,NaN,https://www.amazon.com/dp/1668023369?tag=thene...,NaN,0,Dawn Staley,https://static01.nyt.com/bestsellers/images/97...,500,329,NaN,nyt://book/93e1c5cf-1a3f-5ede-b081-ca9bbd7d30a2,...,2,0,NaN,UNCOMMON FAVOR,2025-06-02T23:13:46.873Z,1,Biography & Autobiography,2025-06-08,NaN,NaN
2,NaN,https://www.amazon.com/dp/0525561722?tag=thene...,NaN,0,Ron Chernow,https://static01.nyt.com/bestsellers/images/97...,500,329,NaN,nyt://book/b92f68c9-76ad-5510-8520-c5864d663b19,...,3,1,NaN,MARK TWAIN,2025-06-02T23:13:46.947Z,2,Biography & Autobiography,2025-06-08,NaN,NaN
3,NaN,https://www.amazon.com/dp/1668053373?tag=thene...,NaN,0,Patrick McGee,https://static01.nyt.com/bestsellers/images/97...,500,331,NaN,nyt://book/82ca74aa-bab8-5b05-9a1f-f8f7e383e1e2,...,4,0,NaN,APPLE IN CHINA,2025-06-02T23:13:47.02Z,1,Business & Economics,2025-06-08,NaN,NaN
4,NaN,https://www.amazon.com/dp/0593655036?tag=thene...,NaN,0,Jonathan Haidt,https://static01.nyt.com/bestsellers/images/97...,500,329,NaN,nyt://book/7557cf43-7888-5c15-8206-d3541cccd89b,...,5,2,NaN,THE ANXIOUS GENERATION,2025-06-02T23:13:47.093Z,61,Psychology,2025-06-08,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3035,NaN,http://www.amazon.com/Fifty-Shades-Trilogy-Bun...,NaN,0,E. L. James,https://static01.nyt.com/bestsellers/images/97...,197,128,NaN,nyt://book/dd89aefc-ae79-5850-ac96-f940ba2cc4fb,...,16,0,NaN,FIFTY SHADES TRILOGY,2025-05-14T21:54:40.666Z,0,NaN,2012-08-12,NaN,NaN
3036,NaN,http://www.amazon.com/The-Marriage-Bargain-Bil...,NaN,0,Jennifer Probst,https://static01.nyt.com/bestsellers/images/A0...,203,128,NaN,nyt://book/cac1eae6-106b-5a28-88cf-73cdf4dd4471,...,17,0,NaN,THE MARRIAGE BARGAIN,2025-05-14T06:55:56.627Z,0,NaN,2012-08-12,NaN,NaN
3037,NaN,http://www.amazon.com/The-Litigators-Novel-Joh...,NaN,0,John Grisham,https://static01.nyt.com/bestsellers/images/97...,229,128,NaN,nyt://book/7c934be9-85db-5ab2-b6a8-89c07600fa24,...,18,0,NaN,THE LITIGATORS,2025-05-14T06:55:56.769Z,0,NaN,2012-08-12,NaN,NaN
3038,NaN,http://www.amazon.com/Backfire-FBI-Thriller-Ca...,NaN,0,Catherine Coulter,https://static01.nyt.com/bestsellers/images/97...,495,327,NaN,nyt://book/c3da01f5-1a39-5460-a8b5-ae4e2e5015bf,...,19,0,NaN,BACKFIRE,2025-05-14T06:55:56.698Z,0,NaN,2012-08-12,NaN,NaN


<!-- This is kind of interesting, but it doesn't show change over time. I've decided to use the NYT best sellers list, specifically non-fiction becuase I'm able to crossreference more specific categories from the Google Books API. The question becomes:

**How have American tastes in non-fiction changed over time?** -->

In [73]:
# Able to calculate date as loop through calls. Neet to remember to always call strftime.
from datetime import datetime
from datetime import timedelta, date
import dateutil.relativedelta


date = datetime.now()
print(date.strftime("%Y-%m-%d"))

while (date.weekday() != 6):
    date = date - timedelta(1)

print(date)
pub_date = date.strftime("%Y-%m-%d")
print(f"pubdate: {pub_date}")
date.weekday()


2025-06-17
2025-06-15 20:32:07.528287
pubdate: 2025-06-15


6

In [156]:
# Be sure to test that this works before running the full pull! Google Books limits api at 1000 per day.


def get_category(isbn):
    if (len(isbn)==13):
        # First, check cache
        try:
            category = df_cached[df_cached['primary_isbn13'] == isbn]['category'].iloc[0]
            print('isbn from cache')
            return category
        
        # If not in cache, call Google Books API
        except:

            response = requests.get(f'https://books.googleapis.com/books/v1/volumes?q=isbn%3A{isbn}&key={google_key}')
            data = response.json()

            if(response.status_code != 200):
                print(response.status_code)
                print(response.headers)

            try:
                category = data['items'][0]['volumeInfo']['categories'][0]
                print('isbn from api')
                return category
            except:
                category = data['items'][0]['volumeInfo']['categories']
                print('isbn from api')
                return category


# uncomment to test   
get_category('9798217060672')

isbn from cache


'Political Science'

In [75]:
# Get NYT

# This now returns an error, as the interval now runs longer than there is data to call. You might figure out a way to bypass this with except while still telling you what the error is.

dfs = []

weeks = 120
time_interval = 30

# start_date = '03/28/2025'
# start_date = datetime.strptime(start_date, '%m%d,%y')

# pub_date = start_date

for week in range(1,weeks+1):
    print(pub_date)
     # First, check cache
    if (df_cached[df_cached['published_date']==pub_date].shape[0] > 1):
            df = df_cached[df_cached['published_date']==pub_date]
            print('nyt list from cache')
            
    
    # If not in cache, call Google Books API
    else:
        try:
            response = requests.get(f'https://api.nytimes.com/svc/books/v3/lists/overview.json?api-key={nyt_key}&published-date={pub_date}')
            print('nyt list from api')

            if(response.status_code != 200):
                print(response.status_code)
                print(response.headers)
            data = response.json()['results']

            nf_combined=data['lists'][1]['books']

            df = pd.DataFrame(nf_combined)


            # Add categories by calling Google Books API
            df['category'] = df['primary_isbn13'].apply(get_category)
            df['published_date'] = data['published_date']

            # NYT API call limit of 5 per minute, 400 per day.
            time.sleep(12)
        except:
            if(response.status_code != 200):
                print(response.status_code)
                print(response.headers)
    
    df['date_datetime']=date
    dfs.append(df)


    date = (date - timedelta(time_interval))
    while (date.weekday() != 6):
        date = date - timedelta(1)
    pub_date = date.strftime("%Y-%m-%d")



2025-06-15
nyt list from cache
2025-05-11
nyt list from cache
2025-04-06
nyt list from cache
2025-03-02
nyt list from cache
2025-01-26
nyt list from cache
2024-12-22
nyt list from cache
2024-11-17
nyt list from cache
2024-10-13
nyt list from cache
2024-09-08
nyt list from cache
2024-08-04
nyt list from cache
2024-06-30
nyt list from cache
2024-05-26
nyt list from cache
2024-04-21
nyt list from cache
2024-03-17
nyt list from cache
2024-02-11
nyt list from cache
2024-01-07
nyt list from cache
2023-12-03
nyt list from cache
2023-10-29
nyt list from cache
2023-09-24
nyt list from cache
2023-08-20
nyt list from cache
2023-07-16
nyt list from cache
2023-06-11
nyt list from cache
2023-05-07
nyt list from cache
2023-04-02
nyt list from cache
2023-02-26
nyt list from cache
2023-01-22
nyt list from cache
2022-12-18
nyt list from cache
2022-11-13
nyt list from cache
2022-10-09
nyt list from cache
2022-09-04
nyt list from cache
2022-07-31
nyt list from cache
2022-06-26
nyt list from cache
2022-05-

C:\Users\nmuff\AppData\Local\Temp\ipykernel_19000\988167329.py:50: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [77]:
df = pd.concat(dfs)
df.dropna(subset='category', inplace=True)
df['category']=df['category'].str.lower()

df['category']

# df.to_csv('year_sample.csv', index=False)

915             political science
916                    psychology
917                       medical
918     biography & autobiography
919          business & economics
                  ...            
2717    biography & autobiography
2719    biography & autobiography
2721                      history
2731                   psychology
2736                        humor
Name: category, Length: 1780, dtype: object

In [78]:
df.tail()

,age_group,amazon_product_url,article_chapter_link,asterisk,author,book_image,book_image_height,book_image_width,book_review_link,book_uri,...,rank,rank_last_week,sunday_review_link,title,updated_date,weeks_on_list,category,published_date,Unnamed: 0,date_datetime
2717,NaN,http://www.amazon.com/Orange-Is-New-Black-Wome...,NaN,0,Piper Kerman,https://static01.nyt.com/bestsellers/images/97...,440,330,NaN,nyt://book/9709a090-9545-5c6a-bf57-7238fca45ebe,...,18,0,NaN,ORANGE IS THE NEW BLACK,2025-05-14T21:49:01.484Z,0,biography & autobiography,2014-02-23,NaN,2014-02-23 20:32:07.528287
2719,NaN,http://www.amazon.com/Proof-Heaven-Neurosurgeo...,NaN,0,Eben Alexander,https://static01.nyt.com/bestsellers/images/97...,300,300,NaN,nyt://book/d316319f-d0cc-5cdb-9e1c-8596a8897b70,...,20,0,NaN,PROOF OF HEAVEN,2025-05-15T19:17:18.591Z,0,biography & autobiography,2014-02-23,NaN,2014-02-23 20:32:07.528287
2721,NaN,http://www.amazon.com/Killing-Jesus-Bill-OReil...,NaN,0,Bill O'Reilly and Martin Dugard,https://static01.nyt.com/bestsellers/images/97...,495,326,NaN,nyt://book/0f82d5e9-2d0b-59ad-ba0d-9b007e525849,...,2,2,NaN,KILLING JESUS,2025-05-14T08:08:33.379Z,15,history,2014-01-19,NaN,2014-01-19 20:32:07.528287
2731,NaN,http://www.amazon.com/Quiet-Power-Introverts-W...,NaN,0,Susan Cain,https://static01.nyt.com/bestsellers/images/97...,495,329,NaN,nyt://book/ae7263e5-183f-56bc-bd27-8049c85dcb16,...,12,0,NaN,QUIET,2025-05-14T21:49:01.557Z,27,psychology,2014-01-19,NaN,2014-01-19 20:32:07.528287
2736,NaN,http://www.amazon.com/Everyone-Hanging-Without...,NaN,0,Mindy Kaling,https://static01.nyt.com/bestsellers/images/97...,495,321,NaN,nyt://book/3c676dd1-3c82-5712-a3f1-29cca5e015e1,...,17,0,NaN,IS EVERYONE HANGING OUT WITHOUT ME?,2025-05-14T06:35:17.704Z,0,humor,2014-01-19,NaN,2014-01-19 20:32:07.528287


In [79]:
# def cut_strings(x):
#     if (type(x) == str):
#         new_string = x.replace("['",'')
#         new_string = new_string.replace("']",'')
#         return new_string

# df['category'] = df['category'].apply(cut_strings)
# df_cached['category'] = df_cached['category'].apply(cut_strings)

In [80]:
# Combine current df with cached
df_cached = pd.concat([df_cached,df])

try:
    df_cached.drop(axis='columns', columns=['buy_links','isbns'], inplace=True)
except:
    pass


df_cached.drop_duplicates(subset=['published_date','primary_isbn13'], inplace=True)
df_cached.to_csv('df_cached.csv', index=False)


C:\Users\nmuff\AppData\Local\Temp\ipykernel_19000\1974972025.py:2: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [81]:
best_books = df.groupby(['title','category']).size().reset_index(name='count').sort_values(by='count',ascending=False).head(10)

best_books


,title,category,count
592,THE BODY KEEPS THE SCORE,medical,52
181,EDUCATED,biography & autobiography,28
598,THE BOYS IN THE BOAT,sports & recreation,26
337,KILLERS OF THE FLOWER MOON,true crime,26
506,SAPIENS,science,24
267,HILLBILLY ELEGY,social science,24
79,BECOMING,biography & autobiography,22
117,BRAIDING SWEETGRASS,nature,20
461,OUTLIVE,health & fitness,19
91,BETWEEN THE WORLD AND ME,biography & autobiography,18


In [82]:
fig = px.bar(best_books, x='title',y='count', title="Weeks on NYT Best Sellers List", hover_data='category', color='category')
fig.show()

If biographies are not the best books, why are they on the list so often?

In [83]:
top10_cat = df['category'].value_counts().reset_index().head(10)


In [84]:
fig = px.bar(top10_cat, x = 'category', y = 'count')
fig.show()

In [85]:
fig = px.scatter(df[(df['rank']<6)], x = 'published_date', y = 'rank', color='category',opacity=1, hover_data='title',)
fig.show()

In [86]:
top_cats = top10_cat['category'].to_list()

In [87]:
top_cats

['biography & autobiography',
 'history',
 'political science',
 'social science',
 'true crime',
 'science',
 'business & economics',
 'medical',
 'nature',
 'humor']

In [88]:
clean_df = df[['title','published_date','category','rank']]
clean_df.head(31)

,title,published_date,category,rank
915,ORIGINAL SIN,2025-06-15,political science,1
916,THE ANXIOUS GENERATION,2025-06-15,psychology,2
917,THE BODY KEEPS THE SCORE,2025-06-15,medical,3
918,MARK TWAIN,2025-06-15,biography & autobiography,4
919,ABUNDANCE,2025-06-15,business & economics,5
920,APPLE IN CHINA,2025-06-15,business & economics,6
921,THE FATE OF THE DAY,2025-06-15,history,7
922,BIG DUMB EYES,2025-06-15,biography & autobiography,8
923,UNCOMMON FAVOR,2025-06-15,biography & autobiography,9
924,ON TYRANNY,2025-06-15,political science,10


In [89]:
clean_df.to_csv('clean_df.csv',index=False)

Could it be there are just more biographies? This is a 5 year old scrape of 46k books from Goodreads. I can run the ISBNs to get the same category information but this will take a few weeks even if it runs every day due to Google Books API limitation. There is no good way to filter out fiction before running them.

In [90]:
books = pd.read_csv('books.csv')

In [ ]:
sample_cached = pd.read_csv('sample_cache.csv')

In [ ]:
# Be sure to test that this works before running the full pull! Google Books limits api at 1000 per day.


def get_category_by_title(book):

# First, check cache
    try:
        category = sample_cached[sample_cached['isbn'] == book['isbn']]['category'].iloc[0]
        print('isbn from cache')
        return category

    # If not in cache, call Google Books API
    except:
        query =(book['title']+" "+book['authors']).replace(" ","_")
        

        response = requests.get(f'https://books.googleapis.com/books/v1/volumes?q={query}&key={google_key}')
        data = response.json()

        if(response.status_code != 200):
            print(response.status_code)
            print(response.headers)

        try:
            category = data['items'][0]['volumeInfo']['categories'][0]
            print('isbn from api')
            return category
        except:
            category = data['items'][0]['volumeInfo']['categories']
            print('isbn from api')
            return category
        finally:
            pass

# uncomment to test   
test_book = books.iloc[1275]
test_cat = get_category_by_title(test_book)
print(test_cat)


category from API (single)
Fiction


In [172]:
#sample 100 rows to start with

books_sample = books.sample(900)

books_sample['category'] = books_sample.apply(get_category_by_title, axis=1)

category from API (single)
No category found
category from API (single)
category from API (single)
No category found
category from API (single)
category from API (single)
category from API (single)
No category found
category from API (single)
No category found
category from API (single)
category from API (single)
category from API (single)
category from API (single)
category from API (single)
category from API (single)
category from API (single)
category from API (single)
category from API (single)
category from API (single)
category from API (single)
category from API (single)
category from API (single)
category from API (single)
category from API (single)
category from API (single)
category from API (single)
category from API (single)
category from API (single)
category from API (single)
category from API (single)
No category found
category from API (single)
category from API (single)
category from API (single)
category from API (single)
category from API (single)
category from API (

In [ ]:
books_sample['category'].value_counts()

category
Fiction                                 265
Juvenile Fiction                         45
Biography & Autobiography                32
Literary Criticism                       23
Philosophy                               20
                                       ... 
Fantasy fiction                           1
Architecture                              1
Chicago (Ill.)                            1
Sperm whale                               1
Kensington Gardens (London, England)      1
Name: count, Length: 120, dtype: int64

In [ ]:
# Combine current df with cached
sample_cached = pd.concat([sample_cached,books_sample])

sample_cached.drop_duplicates(subset=['isbn'], inplace=True)
sample_cached.to_csv('sample_cached.csv', index=False)
